<a href="https://colab.research.google.com/github/victorlf4/MineRLDecisionTransformers/blob/master/notebooks/Decision_Transformer_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import plotly.io as pio

# Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
pio.renderers.default = "colab"

In [2]:
!pip install git+https://github.com/neelnanda-io/Easy-Transformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/neelnanda-io/Easy-Transformer.git to /tmp/pip-req-build-yr7qgott
  Running command git clone -q https://github.com/neelnanda-io/Easy-Transformer.git /tmp/pip-req-build-yr7qgott
     |████████████████████████████████| 41 kB 286 kB/s 
     |████████████████████████████████| 441 kB 9.2 MB/s 
     |████████████████████████████████| 5.5 MB 51.3 MB/s 
     |████████████████████████████████| 1.9 MB 48.5 MB/s 
     |████████████████████████████████| 237 kB 47.3 MB/s 
     |████████████████████████████████| 115 kB 53.1 MB/s 
     |████████████████████████████████| 163 kB 41.1 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 95 kB 3.2 MB/s 
     |████████████████████████████████| 127 kB 53.3 MB/s 
     |████████████████████████████████| 115 kB 51.7 MB/s 
     |████████████████████████████████| 51 kB 5.9 MB/s 
     |

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

# from google.colab import drive
from pathlib import Path
import pickle
import os


import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

import gym

In [4]:
from easy_transformer.utils import (
  gelu_new,
  to_numpy,
  get_corner,
  lm_cross_entropy_loss,
)  # Helper functions
from easy_transformer.hook_points import (
  HookedRootModule,
  HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig
import easy_transformer
from easy_transformer.experiments import (
  ExperimentMetric,
  AblationConfig,
  EasyAblation,
  EasyPatching,
  PatchingConfig,
)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
print(device)

cuda


Some plotting code. Wrappers around Plotly, not important to understand.

In [7]:
def imshow(tensor, yaxis="", xaxis="", **kwargs):
  tensor = to_numpy(tensor)
  plot_kwargs = {"color_continuous_scale":"RdBu", "color_continuous_midpoint":0.0, "labels":{"x":xaxis, "y":yaxis}}
  plot_kwargs.update(kwargs)
  px.imshow(tensor, **plot_kwargs).show()

In [8]:
def gymEnvSample(env, batch_size=5, sequence_size=10):
  batch = []

  for i in range(batch_size):
    env.reset()

    steps = []
    for j in range(sequence_size):
      action = env.action_space.sample()
      obs, reward, done, info = env.step(action)
      steps.append([reward, obs, action])

    batch.append(steps)

  tensor = torch.as_tensor(batch)
  return tensor.flatten(start_dim=1)

# Multi-armed bandits environment

In [9]:
class MultiarmedBanditsEnv(gym.Env):
  """Environment for multiarmed bandits"""
  metadata = {'render.modes': ['human']}

  def __init__(self, nr_arms=10):
    super(MultiarmedBanditsEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(nr_arms)
    self.observation_space = gym.spaces.Discrete(1)
    self.state = 0
    self.reset()

  def step(self, action):
    assert self.action_space.contains(action)
    reward = self.values[action]

    return self.state, reward, False, {self.optimal}

  def reset(self):
    self.values = list(range(self.action_space.n))
    np.random.shuffle(self.values)
    self.optimal = np.argmax(self.values)
    return self.state

  def render(self, mode='human', close=False):
    print("You are playing a %d-armed bandit" % self.action_space.n)

In [10]:
gymEnvSample(MultiarmedBanditsEnv())

tensor([[7, 0, 1, 7, 0, 1, 0, 0, 3, 4, 0, 0, 8, 0, 4, 5, 0, 6, 3, 0, 8, 5, 0, 6,
         5, 0, 6, 1, 0, 9],
        [6, 0, 7, 6, 0, 7, 5, 0, 3, 3, 0, 0, 7, 0, 5, 0, 0, 6, 0, 0, 6, 8, 0, 4,
         4, 0, 2, 4, 0, 2],
        [8, 0, 5, 3, 0, 0, 9, 0, 8, 2, 0, 6, 1, 0, 9, 0, 0, 2, 3, 0, 0, 2, 0, 6,
         9, 0, 8, 5, 0, 7],
        [2, 0, 4, 0, 0, 6, 2, 0, 4, 0, 0, 6, 5, 0, 5, 1, 0, 9, 8, 0, 3, 4, 0, 8,
         8, 0, 3, 8, 0, 3],
        [9, 0, 8, 3, 0, 5, 2, 0, 9, 5, 0, 3, 9, 0, 8, 9, 0, 8, 2, 0, 9, 0, 0, 4,
         1, 0, 0, 0, 0, 4]])

# Treasure hunt environment

In [11]:
class TreasureHuntEnv(gym.Env):
  """Environment for treasure hunt"""
  metadata = {'render.modes': ['human']}

  def __init__(self, nr_chests=10):
    super(TreasureHuntEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(nr_chests)
    self.observation_space = gym.spaces.Discrete(1)
    self.state = 0
    self.reset()

  def step(self, action):
    assert self.action_space.contains(action)
    reward = self.values[action]

    return self.state, reward, False, {self.optimal}

  def reset(self):
    self.values = np.zeros(self.action_space.n, dtype=np.int32)
    self.values[np.random.randint(0, self.action_space.n)] = 1
    self.optimal = np.argmax(self.values)
    return self.state

  def render(self, mode='human', close=False):
    print("You are playing treasure hunt")

In [12]:
gymEnvSample(TreasureHuntEnv())

tensor([[0, 0, 0, 0, 0, 5, 0, 0, 3, 0, 0, 7, 0, 0, 9, 0, 0, 7, 0, 0, 6, 0, 0, 1,
         0, 0, 3, 0, 0, 0],
        [0, 0, 6, 0, 0, 2, 0, 0, 8, 0, 0, 2, 0, 0, 4, 0, 0, 1, 1, 0, 3, 0, 0, 0,
         0, 0, 0, 0, 0, 7],
        [0, 0, 8, 0, 0, 3, 0, 0, 5, 0, 0, 8, 0, 0, 6, 0, 0, 4, 0, 0, 4, 0, 0, 0,
         0, 0, 1, 0, 0, 1],
        [0, 0, 6, 1, 0, 8, 1, 0, 8, 0, 0, 4, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 4,
         1, 0, 8, 0, 0, 3],
        [0, 0, 8, 0, 0, 2, 0, 0, 7, 1, 0, 6, 0, 0, 8, 0, 0, 2, 0, 0, 4, 0, 0, 9,
         0, 0, 0, 0, 0, 0]])

# Training the model

In [42]:
tiny_cfg = EasyTransformerConfig(
  d_model=32,
  d_head=16,
  n_heads=2,
  d_mlp=128,
  n_layers=2,
  n_ctx=60,
  act_fn="solu_ln",
  d_vocab=10,
  normalization_type="LN",
  seed=23,  # Now we're training a custom model, it's good to set the seed to get reproducible results. It defaults to 42.
)

tiny_model = EasyTransformer(tiny_cfg).to(device)
tiny_optimizer = torch.optim.Adam(tiny_model.parameters(), lr=1e-3)
batch_size = 128
num_epochs = 301
env = TreasureHuntEnv()

assert tiny_model.cfg.n_ctx % 3 == 0, "n_ctx must be divisible by 3"

Moving model to device:  cuda
Moving model to device:  cuda


In [43]:
losses = []

# TODO maybe use a learning rate scheduler
for epoch in tqdm.tqdm(range(num_epochs)):
  batch = gymEnvSample(env, batch_size=batch_size, sequence_size=int(tiny_model.cfg.n_ctx / 3))
  loss = tiny_model(batch, return_type="loss")
  loss.backward()
  tiny_optimizer.step()
  tiny_optimizer.zero_grad()
  losses.append(loss.item())
  if epoch % 100 == 0:
    print(f"Epoch: {epoch}. Loss: {loss}")

px.line(losses, labels={"x": "Epoch", "y": "Loss"})

  0%|          | 0/301 [00:00<?, ?it/s]

Epoch: 0. Loss: 2.252415180206299
Epoch: 100. Loss: 0.9057027101516724
Epoch: 200. Loss: 0.893937349319458
Epoch: 300. Loss: 0.8883473873138428


In [44]:
logits, tiny_cache = tiny_model.run_with_cache(batch)
print("Loss:", losses[-1])
print("Cache:", tiny_cache)

Loss: 0.8883473873138428
Cache: ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_attn', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_mid', 'blocks.0.mlp.ln.hook_scale', 'blocks.0.mlp.ln.hook_normalized', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_attn', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'bloc

In [45]:
with torch.no_grad():
  thing = torch.as_tensor([1,0,0])
  logit = tiny_model(thing)
  print(logit.shape, np.argmax(logit.cpu().numpy(), axis=1).shape)

torch.Size([1, 3, 10]) (1, 10)


# Visualising attention patterns

In [46]:
for layer in range(tiny_model.cfg.n_layers):
  for head in range(tiny_model.cfg.n_heads):
    imshow(to_numpy(tiny_cache[f'blocks.{layer}.attn.hook_attn'].mean(0)[head]), title=f'Layer {layer} Attention Pattern, Head {head}', height=500, width=500, xaxis="Token", yaxis="Activation")

# No quincallada

In [47]:
# Install an older version of node, so that Svelte works (a web dev framework)
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
!pip install git+https://github.com/neelnanda-io/PySvelte.git
import sys
sys.path.append('/content/PySvelte')


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://deb.nodesource.com/node_16.x bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backp

In [96]:
def array2text(array):
  str_array=[]
  assert len(array)% 3 == 0, "array lenght must be divisible by 3"
  for i in range(int(len(array)/3)):
      str_array.append("S"+str(i)+"_R:"+str(array[i]))
      str_array.append("S"+str(i)+"_O:"+str(array[i+1]))
      str_array.append("S"+str(i)+"_A:"+str(array[i+1]))
  return str_array

print(array2text([1,2,3]))



['S0_R:1', 'S0_O:2', 'S0_A:2']


In [98]:
import pysvelte
tokens_test=[1,0,1,0,5,0,1,0,0]
test_tokens_str=array2text(tokens_test)
test_tokens=torch.as_tensor(tokens_test)
logits, tiny_cache = tiny_model.run_with_cache(test_tokens)

for layer in range(tiny_model.cfg.n_layers):
    print("Attention for layer", layer)
    
    print(tiny_cache[f'blocks.{layer}.attn.hook_attn'][0].shape)
    pysvelte.AttentionMulti(attention=tiny_cache[f'blocks.{layer}.attn.hook_attn'][0].permute(1, 2, 0),tokens=test_tokens_str).show() #.attn.hook_attn']

Attention for layer 0
torch.Size([2, 9, 9])


Attention for layer 1
torch.Size([2, 9, 9])


# Quincallada

In [79]:
imshow(to_numpy(tiny_cache["attn", 0].mean([0, 1])), title="Layer 0 Attention Pattern", height=500, width=500)

/usr/local/lib/python3.7/dist-packages/plotly/express/_imshow.py:354: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [80]:
env = TreasureHuntEnv()
for i in range(2000):
  action = int(input("Action: "))
  obs, rewards, done, info = env.step(action)
  print(rewards)

KeyboardInterrupt: ignored